In [1]:
import nltk
import sys 
sys.path.append("../")
from LIB.utils import * 
import re
from sklearn.decomposition  import  PCA
from sklearn.feature_extraction.text import TfidfVectorizer


In [2]:
train=pd.read_json("./train.zip")
test=pd.read_json("./test_questions.zip")

In [3]:
train=train[["id","text"]]
test=test[["id","text"]]

In [4]:
train["text"]=train["text"].apply(lambda x: " ".join(filter(lambda y:(y[0]!='@') & (y[0]!=',@')&(y[0]!='#') & (y[0]!=',')
                                                            , x.split())))
train["text"]=train["text"].apply(lambda  x: re.sub(r'http\S+', '', x,flags=re.MULTILINE) ) 

tokenizer=nltk.tokenize.WordPunctTokenizer()
train["text"]=tokenizer.tokenize_sents(train["text"])
# train["text"]=train["text"].apply(lambda x : " ".join(x))
stemmer= nltk.stem.WordNetLemmatizer()
train["text"]=train["text"].apply(lambda x : " ".join(stemmer.lemmatize(token) for token in x ) ) 


train["text"]=train["text"].apply(lambda x: " ".join(filter(lambda y:(y[0]!='@') & (y[0]!=',@')&(y[0]!='#') & (y[0]!=',')
                                                            , x.split())))

train["text"]=train["text"].apply(lambda x :''.join(i for i in x if not i.isdigit()))

tokenizer=nltk.tokenize.WordPunctTokenizer()
train["text"]=tokenizer.tokenize_sents(train["text"])
train["text"]=train["text"].apply(lambda x : " ".join(x))

test["text"]=test["text"].apply(lambda x: " ".join(filter(lambda y:(y[0]!='@') & (y[0]!=',@')&(y[0]!='#') & (y[0]!=',')
                                                            , x.split())))
test["text"]=test["text"].apply(lambda  x: re.sub(r'http\S+', '', x,flags=re.MULTILINE) ) 

tokenizer=nltk.tokenize.WordPunctTokenizer()
test["text"]=tokenizer.tokenize_sents(test["text"])
# test["text"]=test["text"].apply(lambda x : " ".join(x))
stemmer= nltk.stem.WordNetLemmatizer()
test["text"]=test["text"].apply(lambda x : " ".join(stemmer.lemmatize(token) for token in x ) ) 


test["text"]=test["text"].apply(lambda x: " ".join(filter(lambda y:(y[0]!='@') & (y[0]!=',@')&(y[0]!='#') & (y[0]!=',')
                                                            , x.split())))

test["text"]=test["text"].apply(lambda x :''.join(i for i in x if not i.isdigit()))

tokenizer=nltk.tokenize.WordPunctTokenizer()
test["text"]=tokenizer.tokenize_sents(test["text"])
test["text"]=test["text"].apply(lambda x : " ".join(x))

In [6]:
train["train"]=1
test["train"]=0

data=pd.concat([train,test])

In [7]:
del train ,test 
gc.collect()

14

In [8]:
vectorizer = TfidfVectorizer()
corpus=data.text.values.tolist()
X = vectorizer.fit(corpus)

In [9]:
tf_idf=X.transform(data["text"])
tf_idf_DF=pd.DataFrame()
pca_tf_idf=PCA(n_components=20)
tf_idf_array=pca_tf_idf.fit_transform(tf_idf.toarray())
for i in range(tf_idf_array.shape[1]) :
    tf_idf_DF["type_tf_idf_pca_"+str(i)]=tf_idf_array[:,i]

In [10]:
final_data=pd.concat([data.reset_index(),tf_idf_DF],axis=1)

In [11]:
final_data.drop(["index","text"],axis=1).to_csv("../data_proc/TF_IDF_20.csv",index=False)